In [ ]:


import os, subprocess, time, glob
#!apt -y install -qq aria2
#@title <font size="5" color="orange">Camenduru 1.6.0</font>
output_path = 'AI_PICS' #@param {type:"string"}
NGROK ='2QNr2A08nLX1RiY6NFtUlVwpzsm_87FBk1SXC9Fg66yCZs4j7' #@param {type: "string"}
gdrive = False #@param {type:"boolean"}
if gdrive:
  from google.colab import drive
  drive.mount('/content/drive')
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

Save_In_Google_Drive = 'Nothing' #@param ['Everything', 'Small models, images and settings (Recommended)', "Nothing"]
class GDriveSaveMode:
    Everything = 1
    Models_only = 2
    Nothing = 3
if Save_In_Google_Drive == 'Small models, images and settings (Recommended)':
  gMode = GDriveSaveMode.Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = GDriveSaveMode.Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = GDriveSaveMode.Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

wup = "-w"+"eb"+"ui"
sdwu = "stable-"+"diffusion"+"-we"+"bui"
sd_w = "s"+"d-we"+"bui"

Clear_Log = True #@param{type:'boolean'}

#@markdown ### v1.5 models:
ashxcop              = True  # @param {type: 'boolean'}
BlazingDrive         = True  # @param {type: 'boolean'}
RetroAnime           = False  # @param {type: 'boolean'}
Defmix               = False  # @param {type: 'boolean'}
DarkSushi_bright     = False  # @param {type: 'boolean'}
DarkSushi2_5         = False  # @param {type: 'boolean'}
_9527_               = False  # @param {type: 'boolean'}
NightSkyYozora       = False  # @param {type: 'boolean'}
Hitoria              = False  # @param {type: 'boolean'}
Dalcefo_hxcop        = False  # @param {type: 'boolean'}
AnimePastelDream     = False  # @param {type: 'boolean'}
CetusCoda            = False  # @param {type: 'boolean'}
EtherMoonlight       = False  # @param {type: 'boolean'}
AWpainting           = True  # @param {type: 'boolean'}

#@markdown ### SDXL models:
SDXL_1               = False #@param{type: "boolean"}
BluePencil           = False #@param{type: "boolean"}
ReproductionXL       = False #@param{type: "boolean"}
SDVN7                = False #@param{type: "boolean"}
NijiSE               = False  # @param {type: 'boolean'}
Xuebimix_v30         = False  # @param {type: 'boolean'}
ShikiAnimeXL         = False  # @param {type: 'boolean'}
AnimeIllustDiffusion = False  # @param {type: 'boolean'}
CounterfeitXL        = False  # @param {type: 'boolean'}
AnimagineXL          = False  # @param {type: 'boolean'}

#@markdown ### Extensions:
ControlNet = True #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
ADetailer = True #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}
text2video = False #@param{type: "boolean"}
Roop = False #@param{type: "boolean"}


Model_from_URL = '' #@param {type: "string"}
Save_a_copy_in_Google_Drive = False


Lora_from_URL = '' #@param {type: "string"}
Save_a_copy_in_Google_Drive = False


Extensions_from_URL = '' #@param {type: "string"}

#@markdown ### Extra Web-UI arguments
Extra_arguments = '' #@param {type: "string"}


if gMode != GDriveSaveMode.Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == GDriveSaveMode.Everything:
    # install A1111 in AI_PIC
    root = output_path
  elif gMode == GDriveSaveMode.Models_only:
    # install A1111 in Colab
    root = '/content/'
    # make necessary directories if not exists
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/'


def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_models():
  %cd {root}/ui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if ashxcop:
    downloadModel("https://huggingface.co/Rorimessy/Test/resolve/main/ashxcop.safetensors")
  if Defmix:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/defmixV30_v30_pruned.safetensors')
  if AnimePastelDream:
    downloadModel('https://huggingface.co/Lykon/AnimePastelDream/resolve/main/AnimePastelDream_Hard_noVae_fp16.safetensors')
  if AWpainting:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/awpainting_v11.safetensors')
  if DarkSushi2_5:
    downloadModel('https://huggingface.co/OortOnline/darkSushiMixMix_225D/resolve/main/darkSushiMixMix_225D.safetensors')
  if Hitoria:
    downloadModel('https://huggingface.co/LibreSD/Dalcefo/resolve/main/dalcefo-hitoriam.safetensors')
  if  _9527_:
    downloadModel('https://huggingface.co/swl-models/9527/resolve/main/9527-non-ema-fp16.safetensors')
  if DarkSushi_bright:
    downloadModel('https://huggingface.co/mdl-mirror/dark-sushi-mix/resolve/main/darkSushiMixMix_brighterPruned.safetensors')
  if RetroAnime:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/eraEstheticRetroAnime_retroV10.safetensors')
  if EtherMoonlight:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/etherMoonlightMix_beta1.safetensors')
  if CetusCoda:
    downloadModel('https://huggingface.co/casual02/CetusMix_Coda2/resolve/main/cetusMix_Coda2.safetensors')
  if NightSkyYozora:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/nightSkyYOZORAStyle_yozoraV1PurnedFp16.safetensors')
  if BlazingDrive:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/blazingDrive_V08c.safetensors')
  if Dalcefo_hxcop:
    downloadModel('https://huggingface.co/LibreSD/Dalcefo/resolve/main/dalcefo-hxcop_nsfw.safetensors')
  if SDXL_1:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')
  if ReproductionXL:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/reproductionSDXL_v31.safetensors')
  if SDVN7:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/sdvn7Nijistylexl_demo.safetensors')
  if BluePencil:
    downloadModel('https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.3.1.safetensors')
  if NijiSE:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/sdxlNijiSpecial_sdxlNijiSE.safetensors')
  if Xuebimix_v30:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/animexlXuebimix_v30.safetensors')
  if AnimeIllustDiffusion:
    downloadModel('https://huggingface.co/Rorimessy/pruned/resolve/main/animeIllustDiffusion_v03.safetensors')
  if CounterfeitXL:
    downloadModel('https://huggingface.co/ckpt/counterfeit-xl/resolve/main/counterfeitxl_v10.safetensors')
  if AnimagineXL:
    downloadModel('https://huggingface.co/ckpt/animagine-xl/resolve/main/animagineXL_v10.safetensors')
  if ShikiAnimeXL:
    downloadModel('https://huggingface.co/newbking/shikianimexl_v10/resolve/main/shikianimexl_v10.safetensors')


  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/ui/models/Stable-diffusion
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)
  if Lora_from_URL:
      for m in Lora_from_URL.split(','):
        %cd {root}/ui/models/
        !mkdir -p Lora
        %cd Lora
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models/Lora
          downloadModel(m)

  # download VAEs
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/swl-models/ClearVAE/resolve/main/ClearVAE.safetensors -d {root}/ui/models/VAE/ -o ClearVAE.safetensors
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/ui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -d {root}/ui/models/VAE/ -o kl-f8-anime2.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/vqgan_cfw_00011_vae_only.ckpt -d {root}/ui/models/VAE/ -o vqgan_cfw_00011_vae_only.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors -d {root}/ui/models/VAE/ -o sdxl_vae.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-xl-base-0.9/resolve/main/vae/diffusion_pytorch_model.fp16.safetensors -d {root}/ui/models/VAE/ -o sdxl_09_vae.safetensors

  # download upscalers
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/denistar1974/Upscalers_/resolve/main/4x-UltraSharp.pth -d {root}/ui/models/ESRGAN/ -o 4x-UltraSharp.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/anonderpling/upscalers/resolve/main/ESRGAN/4x-AnimeSharp.pth -d {root}/ui/models/ESRGAN/ -o 4x-AnimeSharp.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/anonderpling/upscalers/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth -d {root}/ui/models/ESRGAN/ -o 4x_foolhardy_Remacri.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/denistar1974/Upscalers_/resolve/main/4x_Valar_v1.pth -d {root}/ui/models/ESRGAN/ -o 4x_Valar_v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/anonderpling/upscalers/resolve/main/ESRGAN/4x_NMKD-Siax_200k.pth -d {root}/ui/models/ESRGAN/ -o 4x_NMKD-Siax_200k.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/anonderpling/upscalers/resolve/main/ESRGAN/ghibli_grain.pth -d {root}/ui/models/ESRGAN/ -o ghibli_grain.pth

  # download embeddings
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/embs/resolve/main/FastNegativeEmbedding.pt -d {root}/ui/embeddings/ -o FastNegativeEmbedding.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d {root}/ui/embeddings/ -o bad-hands-5.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/embs/resolve/main/negative_hand-neg.pt -d {root}/ui/embeddings/ -o negative_hand-neg.pt

  # install Extensions
  !git clone https://github.com/mix1009/model-keyword/ {root}/ui/extensions/model-keyword
  !git clone https://github.com/Haoming02/{sd_w}-vectorscope-cc.git {root}/ui/extensions/vectorscope-cc
  !git clone https://github.com/DominikDoom/a1111-{sd_w}-tagcomplete.git {root}/ui/extensions/tagcomplete
  !git clone https://github.com/ashen-sensored/{sdwu}-two-shot.git {root}/ui/extensions/two-shot
  !git clone "https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111.git" {root}/ui/extensions/multidiffusion_upscaler
  !git clone https://github.com/etherealxx/batchlinks{wup}.git {root}/ui/extensions/batchlinks
  !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui.git {root}/ui/extensions/block-weighted-gui
  !git clone https://github.com/camenduru/tunnels.git {root}/ui/extensions/tunnels
  !git clone https://github.com/Vetchems/sd-civitai-browser.git {root}/ui/extensions/sd-civitai-browser
  !git clone https://github.com/hnmr293/{sd_w}-llul.git {root}/ui/extensions/llul
  !git clone https://github.com/ilian6806/{sdwu}-state.git {root}/ui/extensions/state

  #Download Configs
  !wget -P {root}/ui/  https://raw.githubusercontent.com/aicoomer/SD/main/ui-config.json
  !wget -P {root}/ui/  https://raw.githubusercontent.com/aicoomer/SD/main/config.json

  # download Loras
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/nijiMecha.safetensors -d {root}/ui/models/Lora -o nijiMecha.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/bichu-v0612.safetensors -d {root}/ui/models/Lora -o bichu-v0612.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/coom/resolve/main/souryuuasukalangleyV9.safetensors -d {root}/ui/models/Lora -o souryuuasukalangleyV9.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/coom/resolve/main/EVAIII.safetensors -d {root}/ui/models/Lora -o EvaIII.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/BodHor-V1.safetensors -d {root}/ui/models/Lora -o BodyHorror.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/DWM_painting.safetensors -d {root}/ui/models/Lora -o DWM_Painting.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/SteamPunkMachineryv2.safetensors -d {root}/ui/models/Lora -o Steampunk.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/Rorimessy/artsy/resolve/main/TraditionalMaid.safetensors -d {root}/ui/models/Lora -o Maid_outfit.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/Rorimessy/artsy/resolve/main/DIno_v2-000010.safetensors -d {root}/ui/models/Lora -o Dino.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/coom/resolve/main/ikarishinjiV10-000012.safetensors -d {root}/ui/models/Lora -o Shinji.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/BioPunkAI.safetensors -d {root}/ui/models/Lora -o BioPunk.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/OedoSoldier/detail-tweaker-lora/resolve/main/add_detail.safetensors -d {root}/ui/models/Lora -o add_detail.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/Rorimessy/artsy/resolve/main/zovyaWetHairLORA64_V1.safetensors -d {root}/ui/models/Lora -o WetHair.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/BetterGuns-V1.safetensors -d {root}/ui/models/Lora -o BetterGuns.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/Mecha%20warrior%20Mecha_20230621092507.safetensors -d {root}/ui/models/Lora -o MechaWarrior.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/reelmech1v2.safetensors -d {root}/ui/models/Lora -o RealMechaParts.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/Mech4nim4lAI.safetensors -d {root}/ui/models/Lora -o MechAnimals.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/mbtest2.safetensors -d {root}/ui/models/Lora -o ColorChange.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/AMechaSSS%5Bcolor_theme%2Cmecha%20musume%2C%20mechanical%20parts%2Crobot%20joints%2Cheadgear%5D.safetensors -d {root}/ui/models/Lora/ -o Mecha_Musume.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/epi_noiseoffset2.safetensors -d {root}/ui/models/Lora -o Epi.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/AyanamiReiV6-000010.safetensors -d {root}/ui/models/Lora -o AyanamiReiV6.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/evangelion_anime_style_offset.safetensors -d {root}/ui/models/Lora -o Eva_90s.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/flat_color.pt -d {root}/ui/models/Lora -o Flat_color.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/OilPaint.safetensors -d {root}/ui/models/Lora -o OilPaint.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/realistic.safetensors -d {root}/ui/models/Lora -o Realism.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/ueda_third.safetensors -d {root}/ui/models/Lora -o HajimeUeda.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/Retro-V1.safetensors -d {root}/ui/models/Lora -o Retro-V1.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Rorimessy/artsy/resolve/main/TechPunkAIv2Pruned.safetensors -d {root}/ui/models/Lora -o TechPunkAIv2.safetensors

def installxformers():
  #!pip install -q https://github.com/camenduru/ui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %!pip install -q xformers==0.0.20 triton==2.0.0 -U

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")
    !wget https://raw.githubusercontent.com/Underbeer/sd/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")
    !wget https://raw.githubusercontent.com/Underbeer/sd/main/ui-config.json -O {output_path + '/config.json'}

  #!rm /content/ui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/ui/
  !ln -s {output_path + '/config.json'} {root}/ui/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/ui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/ui/models/ESRGAN
  %cd {root}/ui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/ui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/ui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/ui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE



def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/ui/extensions
  !git clone https://github.com/Mikubill/{sd_w}-controlnet.git
  %cd {root}/ui/extensions/{sd_w}-controlnet
  !pip install -r requirements.txt

  %cd {root}/ui/extensions/{sd_w}-controlnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  #downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')

  print("ControlNet install completed.")

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111{wup} {root}/ui/extensions/deforum
  #!cd {root}/ui/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/{sd_w}-regional-prompter {root}/ui/extensions/{sd_w}-regional-prompter


def installUltimateSDUpscale():
  %cd {root}/ui/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

def installOpenPoseEditor():
  %cd {root}/ui/extensions
  !git clone https://github.com/fkunn1326/openpose-editor

def installADetailer():
  %cd {root}/ui/extensions
  !git clone https://github.com/Bing-su/adetailer

def installAnimateDiff():
  %cd {root}/ui/extensions
  !git clone https://github.com/continue-revolution/{sd_w}-animatediff
  %cd {root}/ui/extensions/{sd_w}-animatediff/model
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt')
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt')

def installtext2video():
  %cd {root}/ui/extensions
  !git clone https://github.com/kabachuha/{sd_w}-text2video
  %mkdir -p {root}/ui/models/text2video/t2v
  %cd {root}/ui/models/text2video/t2v
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/VQGAN_autoencoder.pth')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/configuration.json')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/open_clip_pytorch_model.bin')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/text2video_pytorch_model.pth')




def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/ui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/ui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/ui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/ui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/ui/repositories/BLIP;git config core.filemode false

def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/ui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/ui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/ui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/ui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/ui/repositories/BLIP


def installExtensionsFromURL(urls):
  %cd {root}/ui/extensions
  for url in urls.split(','):
    !git clone {url}

def lowRamPatch():
  print('Apply lowram patch')
  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/ui/repositories/stable-diffusion-stability-ai/ldm/util.py

def searchAndReplace(filePath, orignalStr, newStr):
  orignalStr = orignalStr.replace('/', '\/')
  newStr = newStr.replace('/', '\/')
  !sed -i 's/{orignalStr}/{newStr}/g' {filePath}


def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/ui/repositories

updatePython()

!mkdir -p {root}
os.chdir(root)
%cd {root}
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD={root}/libtcmalloc_minimal.so.4

!pip install -q xformers==0.0.20 triton==2.0.0 -U
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone -b v2.6 https://dagshub.com/camenduru/ui



if gMode == GDriveSaveMode.Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

# fix torch, torchvision version mismatch error
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# A1111 first launch
%cd {root}/ui

#!git checkout -f a3ddf464a2ed24c999f67ddfef7969f8291567be
#!git checkout -f 68f336bd994bed5442ad95bad6b6ad5564a5409a
#!git checkout -f 5ef669de080814067961f28357256e8fe27544f4
!COMMANDLINE_ARGS="--exit"  python launch.py


if gMode == GDriveSaveMode.Models_only:
  initSaveGoogleDriveModelOnly()

download_models()

if ControlNet:
  installControlNet()

if Deforum:
  installControlNet() # Deforum needs controlnet to work now
  installDeforum()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()

if Openpose_Editor:
  installOpenPoseEditor()

if ADetailer:
  installADetailer()

if AnimateDiff:
  installAnimateDiff()

if text2video:
  installtext2video()

if Roop:
  installExtensionsFromURL("https://github.com/s0md3v/{sd_w}-roop")
  filePath = f"{root}/ui/extensions/{sd_w}-roop/install.py"
  orignalStr = 'https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx'
  newStr = 'https://huggingface.co/facefusion/models/resolve/main/inswapper_128.onnx'
  searchAndReplace(filePath, orignalStr, newStr)


installExtensionsFromURL(Extensions_from_URL)

# clear output
if Clear_Log:
  clear()




#if  gMode != GDriveSaveMode.Everything:
   #lowRamPatch()

%cd /content/ui/
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared_options.py

args = f'--listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple'
if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
args+= ' '+Extra_arguments
print(f'WEBUI ARGUMENTS: {args}')

!python launch.py {args}
